# Online Retail – Customer Segmentation & Recommender System
Romeo Cozac (June, 2018)
___
**The Data**: This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

**Goals**: Through this notebook we aim to better understand customer behaviour as well as trends in sales, and to use this information to answer business questions. We look at new customer aquisition through time and repeat purchase rates, as we want to segment customers based on how much value they are likely to bring in the future. Also, we check which products/categories each one of our segments prefer and which ones are more troublesome. Finally, we build a collaborative filtering based recommender system.
___
**1. Setup and Prerequisites**

**2. Visualizing and Exploring the Data**

## 1. Setup and Prerequisites
As a first step, we download the data under from kaggle into to the `data/` directory

In [ ]:
!kaggle datasets download -d carrie1/ecommerce-data -p data/

Next, we load the modules and dependencies used in the notebook

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from scipy import stats

Finally, we load the data into memory

In [ ]:
df = pd.read_csv('data/data.csv', encoding = "ISO-8859-1")

## 2. Visualizing the Data

Our dataset contains the following information:
- **InvoiceNo**: *(discrete)* `string` — A unique identifier for each purchase made. <br />
  - This variable is generally numeric. When the value starts with a "C", it indicates a cancelled item. <br />
- **StockCode**: *(discrete)* `string` — A unique identifier for the product that was purchased. <br />
  - When the fields contains only letters, it indicates a type of operation instead of a product ID (ie: discount or manual adjustment)
- **Description**: *(discrete)* `string` — Name of the purchased product. <br />
  - In case of a certain operation like discounts, this field will contain additional details to what appears in "StockCode
- **Quantity**: *(continuous)* `int` — How many products were bought on this order. <br />
- **InvoiceDate**: *(continuous)* `date` — When the order was made. <br />
- **UnitPrice**: *(continuous)* `float` — Product price per unit (in pound sterling). <br />
  - This can be negative in case of a cancellation or discount
- **CustomerID**: *(discrete)* `int` — A unique identifier for the customer who made the order. <br />
- **Country**: *(discrete)* `string` — Location of the store.

In [ ]:
df.head()

We can see that our dataset is focused on *products* (order items) instead of orders as there are duplicate `InvoiceNo`s. This means that each row represents a unique product from an order made.

### 2.1. Data Preparation and Feature Engineering
We first cast the type of **InvoiceDate** to `datetime` and add a new field called **RowTotal**, which is the total amount paid on the entry.

In [ ]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["RowTotal"] = df["Quantity"] * df["UnitPrice"]

Order based metrics (instead of order items) is also something we want to look at, so we create an aggregated DataFrame based on orders. We compute the **GrandTotal** for each order and count the number of unique products, as well as the total number of items purchased.

In [ ]:
orders = df.groupby("InvoiceNo") \
    .agg({
        'StockCode':'count',
        'Quantity': 'sum',
        'RowTotal': 'sum',
        'Country': 'first',
        'InvoiceDate': 'first',
        'CustomerID': 'first'}) \
    .reset_index() \
    .rename(columns={
        'StockCode':'ProductsCount', 
        'Quantity': 'ItemsCount', 
        'RowTotal': 'GrandTotal'
    })
    
orders.head()

### 2.2. Empty Values

In [ ]:
df.info()

The dataset contains a little over 540,000 entries (order items). Most variables have a value 100% of the time, except for the **Description**, but especially for the **CustomerID**. It is possible the entries missing a customer ID were submited by guests (people who placed an order without creating an account).

We take a closer look at the **Description** field.

In [ ]:
no_descriptions = df[df["Description"].isnull()]
no_descriptions.head()

Most rows that match this description are free items (**UnitPrice** is 0). We check if this is always the case, which indeed is.

In [ ]:
print('There are %s entries with an empty "Description" and a non-zero "UnitPrice"' % sum(no_descriptions["UnitPrice"] > 0))

### 2.3. Basic Statistics
Knowing the smallest product price, the largest number of items purchased at once or the average sale value might be of interest. To get an accurate estimate we will exclude cancelled orders, free items and outliers (keeping ~99.7% of the values).

In [ ]:
clean_transactions = df[(df["RowTotal"] > 0) & (df["StockCode"].str.contains('\d', regex=True))]
clean_transactions = clean_transactions[["Quantity", "UnitPrice", "RowTotal"]]

clean_orders = orders[orders["GrandTotal"] > 0]
clean_orders = clean_orders[["ProductsCount", "ItemsCount", "GrandTotal"]]

clean_transactions = clean_transactions[(np.abs(stats.zscore(clean_transactions)) < 3).all(axis=1)]
clean_orders = clean_orders[(np.abs(stats.zscore(clean_orders)) < 3).all(axis=1)]

In [ ]:
clean_transactions.describe()

Looking at *order items*:
- We are left with around 520,000 entries
- On average 9-10 items are ordered from each product. However, the median is only 4, suggesting more orders with a lower quantity and a few orders with "extreme" quantities (probably wholesalers);
- The price per unit indicates that the vendor is selling low priced items. It ranges from 4p to little over 16£, with a median close to 2£.

In [ ]:
clean_orders.describe()

Looking at *orders*:
- There are close to 20,000 orders in the dataset
- The order statistics as well as the mean of the "ProductsCount" and "ItemsCount" indicates that the vendor is heavily focused on bulk sales. On average, more than 20 different products are purchased on every order with an average total item count of ~230 per order.
- The average order value is little over 400£, with 75% of the orders being ~150£ or over.

### 2.4. Cancelled Orders
We want to know how many orders are cancelled.

In [ ]:
cancelled_orders = orders[orders['InvoiceNo'].str.contains('C')]
cancelled_orders.head()

Besides the **GrandTotal** the **ItemsCount** is also negative for cancelled orders. This could indicate that a new entry is created for cancellations instead of the old ones being overwritten. We wish to verify this theory.

In [ ]:
aggregate_table = df.copy()
aggregate_table['count_per_customer'] = aggregate_table.groupby('CustomerID')['CustomerID'].transform('count')
aggregate_table['min_per_customer'] = aggregate_table.groupby('CustomerID')['RowTotal'].transform('min')

aggregate_table[
    (aggregate_table['count_per_customer'] == 2) 
    & (aggregate_table['min_per_customer'] < 0)
].sort_values(by=['CustomerID', 'StockCode', 'InvoiceDate']).loc[:, 'InvoiceNo':'CustomerID'].head(n=8)

For ease, order items were filtered down to customers with only 2 row items and at least 1 negative row item. We notice several interesting cases:
- It is confirmed that indeed, new items are added in case of a cancellation instead of old rows being updated;
- Partial cancellations are possible, since the quantities don't always match up;
- Not every cancellation has a matching pair (these could be cancellations of orders placed before 01/12/2010).

In [ ]:
orders_count = pd.DataFrame({'Orders Count': [orders.shape[0] - cancelled_orders.shape[0], cancelled_orders.shape[0]]},
                             index=['Processed Orders', 'Cancelled Orders'])

total_items_count = sum(df['Quantity'])
refunded_items_count = sum(df[df['InvoiceNo'].str.contains('C')]['Quantity']) * -1
items_count = pd.DataFrame({'Items Count': [total_items_count - refunded_items_count, refunded_items_count]},
                             index=['Sold Items', 'Refunded Items'])

fig, axes = plt.subplots(1, 2)
axes[0].axis('off')
axes[1].axis('off')

orders_count.plot(kind='pie', y='Orders Count', autopct='%.2f', ax=axes[0], title='Order Statuses', figsize=(14, 7))
items_count.plot(kind='pie', y='Items Count', autopct='%.2f', ax=axes[1], title='Items Purchased', figsize=(14, 7))

Almost 15% of all orders are cancellations, but only ~5% of items are cancellend (due to partial cancellations)
### 2.5. Geolocation
We want to know how orders are distributed through each country

In [ ]:
orders.plot(column='Country', cmap='Blues', figsize=(7,14), scheme='equal_interval', legend=True)